# TRANSFERT LEARNING USING THE SOUNDNET MODEL ON THE GTZAN DATABASE

### Import

In [125]:
import soundfile, torch
import torchaudio
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt

import os
import glob

import pandas as pd
import numpy as np

## Model
### Here we are defining the model and loading the pre-trained soundnet model

In [4]:
from pytorch_model import SoundNet8_pytorch
from utils import vector_to_scenes,vector_to_obj

## define the soundnet model
model = SoundNet8_pytorch()

## Load the weights of the pretrained model
model.load_state_dict(torch.load('sound8.pth'))

<All keys matched successfully>

### We define the all the data paths

In [99]:
genre_dict = {"blues":0,"classical":1,"country":2,"disco":3,"hiphop":4,"jazz":5,"metal":6,"pop":7,"reggae":8,"rock":9}

# Get all the audio file path
df = pd.read_csv("features_30_sec.csv")
list_filename = list(df.filename)

# Split the list in sublists by genre
list_filepath = []
for i in range(10):
    list_filepath.append(list_filename[100*i: 100*(i+1)])

# Complete all the paths
list_audiofile_path = []

for genre, key in genre_dict.items():
    sublist = []
    
    for filepath in list_filepath[key]:
        if filepath ==
        path = os.path.join(r".\genres_original", genre , filepath )
        sublist.append(path)
        
    list_audiofile_path.append(sublist)
    
    

We want to isolate the fifth layer features and then re-classify them with a classifier method

After testing we find that the dimension of the fifth layer features are (1,256,81,1)

## Functions 

In [67]:
# Collect the features wanted from the soundnet modelling of the given audiofile_path

def get_soundnet_features(audiofile_path, n_feature):
    waveform, sr = torchaudio.load(audiofile_path)
    # Reshape the data to the format expected by the model (Batch, 1, time, 1)
    waveform = waveform.view(1,1,-1,1)

    # Extract the features we want
    features = model.extract_feat(waveform)  #features 7 and 8 are object_pred and scene_pred

    # Get the good features
    feature = features[n_feature]

    return feature

In [131]:
# Extract all the fifth layer features and so, create the dataset we will perform the classification on

def create_dataset(list_of_path, genre_dict, n_feature):
    X = []
    Y = []

    for genre, key in genre_dict.items():
        for path in list_of_path[key]:
            if path != '.\\genres_original\\jazz\\jazz.00054.wav':
                X.append(np.array(get_soundnet_features(path, n_feature-1)))
                Y.append(genre)

    return X, Y


x,y = create_dataset(list_audiofile_path, genre_dict, 5)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (999, 1, 256) + inhomogeneous part.

In [124]:
print("The size of our dataset is " + str(len(x)) + " and there are " + str(len(genre_dict)) + " classes.\nThe format of the input data is " + str((len(x[0][0]),len(x[0][0][0]),len(x[0][0][0][0]))) + "." )

The size of our dataset is 999 and there are 10 classes.
The format of the input data is (256, 81, 1).


In [ ]:
# traintestsplit le dataset
# chosir quelle couche je prend de soundet pour le transfert learning fait
# train un nouveau modèle simple (uen simple classification sur scikit-learn)
# comparer les résultats